In [2]:
%load_ext autoreload
import datetime
# import ete3
import itertools
import json
import logging
import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import typing
# import re
import xml.etree.ElementTree as ET
# import xml.dom.minidom
# import xmlschema

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%autoreload
from maxes.xes_loader2 import XesLoader, XesLog
# from maxes.xes_file import XesFile
# from maxes.analyze_xes import AnalyzeXes
# import maxes.analyze_sequence
# import maxes.serialization.serialize
# import maxes.graphs
import maxes.notebooks.utils
import maxes.utils

In [ ]:
maxes.notebooks.utils.init_notebook() # RUN ONLY ONCE

In [ ]:
import maxes.data.load_files

file_path = "/vt/md/maxes/maxes/data/other/simple.xes"
print(file_path)

log = XesLoader().load(file_path)
log.update_df()
log.df

In [ ]:
%autoreload
from maxes.generators.xes_generator.xes_generator3 import XesGenerator3

generator = XesGenerator3()
generator.fit(log)

In [7]:
generator.generate_traces_count = 1
generated_log = generator.generate()
generated_log.df

,case:concept:name,concept:name,lifecycle:transition,time:timestamp,call centre,location,duration,outcome,org:resource
0,0,incoming claim,complete,1970-01-01 03:30:33.416664+00:00,Sydney,NaN,NaN,NaN,customer
1,0,S check if sufficient information is available,start,1970-01-01 03:30:33.368100+00:00,NaN,Sydney,NaN,NaN,Call Centre Agent Sydney
2,0,S check if sufficient information is available,complete,1970-01-01 03:30:46.515393+00:00,NaN,Sydney,NaN,NaN,Call Centre Agent Sydney
3,0,S register claim,start,1970-01-01 03:30:47.564836+00:00,NaN,NaN,NaN,NaN,Call Centre Agent Sydney
4,0,S register claim,complete,1970-01-01 03:45:46.892096+00:00,NaN,NaN,NaN,NaN,Call Centre Agent Sydney
5,0,determine likelihood of claim,start,1970-01-01 06:50:59.071674+00:00,NaN,NaN,NaN,NaN,Claims handler
6,0,determine likelihood of claim,complete,1970-01-01 06:51:14.942638+00:00,NaN,NaN,NaN,NaN,Claims handler
7,0,end,start,1970-01-01 06:51:14.604215+00:00,NaN,NaN,16380,processed,Claims handler
8,0,end,complete,1970-01-01 06:51:14.553535+00:00,NaN,NaN,16380,processed,Claims handler


In [284]:
generated_log.df.columns

Index(['case:concept:name', 'concept:name', 'lifecycle:transition',
       'time:timestamp', 'call centre', 'location', 'duration', 'outcome',
       'org:resource'],
      dtype='object')

In [ ]:
graph = generator.sequence_graph_
edge = list(graph.edges)[0]
edge

(('incoming claim', 'complete'),
 ('B check if sufficient information is available', 'start'))

In [ ]:
def m1(original_log: pd.DataFrame, generated_log: pd.DataFrame, columns=[]):
    generated_log = generated_log[columns]
    unique_rows = original_log[columns].drop_duplicates()

    for event in generated_log.iterrows():
        is_row_present = event in generated_log

def compute_event_match(a: pd.Series, b: pd.Series, columns: list[str]):

    return sum(
        a[column] == b[column] or
        a[column] != a[column] and b[column] != b[column] # or both nan
        for column in columns
    )

def count_best_match(original_log: pd.DataFrame, generated_log: pd.DataFrame, columns=[]):
    generated_log = generated_log[columns]

    scores = []
    for index_generated_event, generated_event in generated_log.iterrows():
        generated_event_scores = []

        for index_original_event, original_event in original_log.iterrows():
            score = compute_event_match(original_event, generated_event, columns)
            generated_event_scores.append(score)

        best_generated_event_score = max(generated_event_scores)
        scores.append(best_generated_event_score)

    scores = pd.Series(scores)
    normalized_scores = scores / len(columns)
    trace_score = normalized_scores.mean()

    return trace_score

columns = ['concept:name', 'lifecycle:transition', 'call centre', 'location']
count_best_match(log.df, generated_log.traces[0].df, columns)

np.float64(1.0)

In [54]:
log.df[columns].reset_index(drop=True)

,concept:name,lifecycle:transition,call centre,location
0,incoming claim,complete,Brisbane,NaN
1,B check if sufficient information is available,start,NaN,Brisbane
2,B check if sufficient information is available,complete,NaN,Brisbane
3,B register claim,start,NaN,NaN
4,B register claim,complete,NaN,NaN
...,...,...,...,...
46133,S register claim,complete,NaN,NaN
46134,determine likelihood of claim,start,NaN,NaN
46135,determine likelihood of claim,complete,NaN,NaN
46136,end,start,NaN,NaN


In [ ]:
from functools import reduce

original_df = log.df[columns].reset_index(drop=True)
generated_df = generated_log.df[columns].reset_index(drop=True)

generated_event = generated_df.iloc[0]

mask = [original_df[column] == generated_event[column] for column in columns]
# mask = reduce(lambda a, b: a & b, mask)
mask = pd.DataFrame(mask).T
mask
# original_df[generated_event]

,concept:name,lifecycle:transition,call centre,location
0,True,True,False,False
1,False,False,False,False
2,False,True,False,False
3,False,False,False,False
4,False,True,False,False
...,...,...,...,...
46133,False,True,False,False
46134,False,False,False,False
46135,False,True,False,False
46136,False,False,False,False


In [166]:
original_df = log.df[columns].reset_index(drop=True)
generated_df = generated_log.df[columns].reset_index(drop=True)

generated_event = generated_df.iloc[0]

generated_event_df = pd.concat([generated_event] * len(original_df), axis=1).transpose().reset_index(drop=True)
# original_df
# generated_event_df
# original_df.compare(generated_event_df)
# (original_df == generated_event_df) | ((original_df != original_df) & (generated_event_df != generated_event_df))
mask = (original_df == generated_event) | ((original_df != original_df) & (generated_event != generated_event))
mask

,concept:name,lifecycle:transition,call centre,location
0,True,True,False,True
1,False,False,False,False
2,False,True,False,False
3,False,False,False,True
4,False,True,False,True
...,...,...,...,...
46133,False,True,False,True
46134,False,False,False,True
46135,False,True,False,True
46136,False,False,False,True


In [167]:
weights = {column: i + 1 for i, column in enumerate(columns)}
weights

{'concept:name': 1, 'lifecycle:transition': 2, 'call centre': 3, 'location': 4}

In [170]:
pd.DataFrame(np.zeros((3, 4)))

,0,1,2,3
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0


In [180]:
shape = (len(original_df), len(columns))
scores = pd.DataFrame(np.zeros(shape, dtype=int), columns=columns)

for column, weight in weights.items():
    scores[column] = mask[column].map({ True: weight, False: 0 })
    # scores[column] = mask[column].replace({ True: weight, False: 0 })

scores

,concept:name,lifecycle:transition,call centre,location
0,1,2,0,4
1,0,0,0,0
2,0,2,0,0
3,0,0,0,4
4,0,2,0,4
...,...,...,...,...
46133,0,2,0,4
46134,0,0,0,4
46135,0,2,0,4
46136,0,0,0,4


In [182]:
scores_sum = scores[columns].sum(axis=1)
scores_sum

0        7
1        0
2        2
3        4
4        6
        ..
46133    6
46134    4
46135    6
46136    4
46137    6
Length: 46138, dtype: int64

In [183]:
scores_sum.max()

np.int64(10)

In [162]:
mask

,concept:name,lifecycle:transition,call centre,location
0,True,True,False,True
1,False,False,False,False
2,False,True,False,False
3,False,False,False,True
4,False,True,False,True
...,...,...,...,...
46133,False,True,False,True
46134,False,False,False,True
46135,False,True,False,True
46136,False,False,False,True


In [130]:
# mask.where([0.1, 0.2, 0.3, 0.4])
pd.Series([0.1, 0.2, 0.3, 0.4]).transpose()
# pd.Series(0.1, 0.2, 0.3, 0.4)
# mask.mask(mask, pd.Series([0.1, 0.2, 0.3, 0.4]))

0    0.1
1    0.2
2    0.3
3    0.4
dtype: float64

In [117]:
original_df.isna()

,concept:name,lifecycle:transition,call centre,location
0,False,False,False,True
1,False,False,True,False
2,False,False,True,False
3,False,False,True,True
4,False,False,True,True
...,...,...,...,...
46133,False,False,True,True
46134,False,False,True,True
46135,False,False,True,True
46136,False,False,True,True


In [116]:
original_df.isna() == generated_event_df.isna()


,concept:name,lifecycle:transition,call centre,location
0,True,True,True,True
1,True,True,False,False
2,True,True,False,False
3,True,True,False,True
4,True,True,False,True
...,...,...,...,...
46133,True,True,False,True
46134,True,True,False,True
46135,True,True,False,True
46136,True,True,False,True


In [112]:
generated_event_df

,concept:name,lifecycle:transition,call centre,location
0,incoming claim,complete,Sydney,NaN
1,incoming claim,complete,Sydney,NaN
2,incoming claim,complete,Sydney,NaN
3,incoming claim,complete,Sydney,NaN
4,incoming claim,complete,Sydney,NaN
...,...,...,...,...
46133,incoming claim,complete,Sydney,NaN
46134,incoming claim,complete,Sydney,NaN
46135,incoming claim,complete,Sydney,NaN
46136,incoming claim,complete,Sydney,NaN


In [111]:
original_df

,concept:name,lifecycle:transition,call centre,location
0,incoming claim,complete,Brisbane,NaN
1,B check if sufficient information is available,start,NaN,Brisbane
2,B check if sufficient information is available,complete,NaN,Brisbane
3,B register claim,start,NaN,NaN
4,B register claim,complete,NaN,NaN
...,...,...,...,...
46133,S register claim,complete,NaN,NaN
46134,determine likelihood of claim,start,NaN,NaN
46135,determine likelihood of claim,complete,NaN,NaN
46136,end,start,NaN,NaN


In [99]:
generated_event

concept:name            incoming claim
lifecycle:transition          complete
call centre                     Sydney
location                           NaN
Name: 0, dtype: object

In [64]:
pd.DataFrame([[float('0.1')]]).equals(pd.DataFrame([[float('0.1')]]))

True

In [88]:
# pd.Series(float('nan')).mask
# pd.Series(float('1.0')) == float('1.0')
pd.Series(float('nan')) == float('nan')

0    False
dtype: bool

In [81]:
a = pd.DataFrame([[float('0.1'), float('nan')]], columns=['one', 'two'])
# b = pd.DataFrame([[float('0.1')]])
# pd.Series.compare(a['one'], pd.Series(0.1))
pd.Series.equals(pd.Series(float('nan')), float('nan'))

False

In [40]:
column = columns[0]
original_df[column] == generated_event[column]


0     True
1    False
2    False
3    False
4    False
     ...  
4    False
5    False
6    False
7    False
8    False
Name: concept:name, Length: 46138, dtype: bool

np.float64(0.15687538832777032)

In [11]:
generated_log.df[['concept:name', 'lifecycle:transition', 'call centre', 'location']].drop_duplicates()

,concept:name,lifecycle:transition,call centre,location
0,incoming claim,complete,Sydney,NaN
1,S check if sufficient information is available,start,NaN,Sydney
2,S check if sufficient information is available,complete,NaN,Sydney
3,S register claim,start,NaN,NaN
4,S register claim,complete,NaN,NaN
5,determine likelihood of claim,start,NaN,NaN
6,determine likelihood of claim,complete,NaN,NaN
7,end,start,NaN,NaN
8,end,complete,NaN,NaN


In [ ]:
def count_best_match_df(original_log: pd.DataFrame, generated_log: pd.DataFrame, columns=[], weights=None):
    if weights is None:
        weights = {column: 1 for column in columns}

    # Validation
    missing_columns_in_weights = [column for column in columns if column not in weights]
    if len(missing_columns_in_weights):
        raise ValueError(f"Missing columns in weights: {missing_columns_in_weights}")

    original_df = original_log[columns].reset_index(drop=True)
    generated_df = generated_log[columns].reset_index(drop=True)

    shape = (len(original_df), len(columns))

    scores = []
    for generated_event_index, generated_event in generated_df.iterrows():
        mask = (original_df == generated_event) | ((original_df != original_df) & (generated_event != generated_event))

        event_scores = pd.DataFrame(np.zeros(shape, dtype=int), columns=columns)

        for column, weight in weights.items():
            event_scores[column] = mask[column].map({ True: weight, False: 0 })

        event_scores_sum = event_scores[columns].sum(axis=1)

        best_event_score = event_scores_sum.max()
        scores.append(best_event_score)

    generated_log = generated_log[columns]

    scores = pd.Series(scores)
    normalized_scores = scores / len(columns)
    trace_score = normalized_scores.mean()

    return trace_score

columns = ['concept:name', 'lifecycle:transition', 'call centre', 'location']
count_best_match_df(log.df, generated_log.traces[0].df, columns)

np.float64(1.0)

In [ ]:
# V2
def find_matching_events(
        events: pd.DataFrame,
        event: pd.Series,
        identification_columns: list[str]
):
    id_mask = [events[column] == event[column] for column in identification_columns]
    id_mask = reduce(lambda a, b: a & b, id_mask)
    return events[id_mask]

def find_best_score_categorical(
        original_df: pd.DataFrame,
        event: pd.Series,
        identification_columns: list[str],
        weights: pd.Series
):
    matching_events = find_matching_events(original_df, event, identification_columns)

    matching_events_data = matching_events[weights.keys()]
    event_data = event[weights.keys()]

    similarities = (matching_events_data == event_data) | ((matching_events_data != matching_events_data) & (event_data != event_data))

    diffs = ~similarities
    diffs = diffs.astype(int)
    diffs = diffs * weights

    scores = diffs.sum(axis=1) / weights.sum()

    best_score_index = scores.idxmin()

    score = scores.loc[best_score_index]

    return score, diffs, best_score_index


def count_best_match_df(
        original_df: pd.DataFrame,
        generated_df: pd.DataFrame,
        identification_columns: list[str],
        difference_columns: list[str] | None = None,
        weights: pd.Series | dict[str, float] | None = None
):
    if weights is None and difference_columns is None:
        raise ValueError('Either difference_columns or weights must be present')

    if weights is None:
        weights = {column: 1 for column in difference_columns}
    else:
        difference_columns = list(weights.keys())

    weights = pd.Series(weights)

    identification_columns = set(identification_columns)
    difference_columns = set(difference_columns)

    if identification_columns & difference_columns:
        raise ValueError('identification_columns and difference_columns must not intersect')

    all_columns = list(identification_columns | difference_columns)

    original_df = original_df[all_columns].reset_index(drop=True)
    generated_df = generated_df[all_columns].reset_index(drop=True)

    scores = []
    for generated_event_index, generated_event in generated_df.iterrows():
        score, diffs, best_score_index = find_best_score_categorical(original_df, generated_event, identification_columns, weights)

        # print()
        # print()
        # print()
        # print("event")
        # print(generated_event)

        # print()
        # print("best score index")
        # print(best_score_index)

        # print()
        # print("matched event")
        # print(original_df.iloc[best_score_index])

        scores.append(score)

    mean_score = pd.Series(scores).mean()

    return mean_score

columns = ['concept:name', 'lifecycle:transition', 'call centre', 'location']
count_best_match_df(
    log.df,
    generated_log.traces[0].df,
    identification_columns=['concept:name', 'lifecycle:transition'],
    difference_columns=['call centre', 'location', 'outcome', 'org:resource']
)

np.float64(0.0)

In [242]:
processed_original_df = log.df.copy(deep=True)
processed_original_df['time:timestamp'] -= processed_original_df['time:timestamp'].min()
processed_original_df['time:timestamp'] = processed_original_df['time:timestamp'].dt.total_seconds()
processed_original_df

,case:concept:name,concept:name,lifecycle:transition,time:timestamp,call centre,org:resource,location,duration,outcome,case:description
0,1,incoming claim,complete,0.0,Brisbane,customer,NaN,NaN,NaN,Simulated process instance
1,1,B check if sufficient information is available,start,0.0,NaN,Call Centre Agent Brisbane,Brisbane,NaN,NaN,Simulated process instance
2,1,B check if sufficient information is available,complete,41.0,NaN,Call Centre Agent Brisbane,Brisbane,NaN,NaN,Simulated process instance
3,1,B register claim,start,41.0,NaN,Call Centre Agent Brisbane,NaN,NaN,NaN,Simulated process instance
4,1,B register claim,complete,779.0,NaN,Call Centre Agent Brisbane,NaN,NaN,NaN,Simulated process instance
...,...,...,...,...,...,...,...,...,...,...
4,3512,S register claim,complete,9571.0,NaN,Call Centre Agent Sydney,NaN,NaN,NaN,Simulated process instance
5,3512,determine likelihood of claim,start,24964.0,NaN,Claims handler,NaN,NaN,NaN,Simulated process instance
6,3512,determine likelihood of claim,complete,24995.0,NaN,Claims handler,NaN,NaN,NaN,Simulated process instance
7,3512,end,start,24995.0,NaN,Claims handler,NaN,17306,not liable,Simulated process instance


In [ ]:
def find_matching_events(
        events: pd.DataFrame,
        event: pd.Series,
        identification_columns: list[str]
):
    id_mask = [events[column] == event[column] for column in identification_columns]
    id_mask = reduce(lambda a, b: a & b, id_mask)
    return events[id_mask]

def find_best_score(
        original_df: pd.DataFrame,
        event: pd.Series,
        identification_columns: list[str],
        difference_weights: pd.Series
):
    matching_events = find_matching_events(original_df, event, identification_columns)

    column_diffs = matching_events[difference_weights.keys()] - event[difference_weights.keys()]
    column_diffs = column_diffs.abs()

    # Normalize within each column's range
    column_diffs = column_diffs / column_diffs.max()

    column_diffs = column_diffs * difference_weights

    scores = column_diffs.sum(axis=1) / difference_weights.sum()
    best_score_index = scores.idxmin()

    # Note: best_score_index may be greater than len(scores)
    # because best_score_index is index of original_df, not matching_events
    # It is important to use .loc and not .iloc
    score = scores.loc[best_score_index]

    return score, column_diffs, best_score_index

def count_best_match_numerical(
        original_log: pd.DataFrame,
        generated_log: pd.DataFrame,
        identification_columns: list[str],
        difference_weights: dict[str, float]
):
    difference_columns = set(difference_weights.keys())
    identification_columns = set(identification_columns)

    if identification_columns & difference_columns:
        raise ValueError('identification_columns and difference_weights.keys() must not intersect')

    all_columns = list(set(identification_columns) | set(difference_columns))

    original_df = original_log[all_columns].reset_index(drop=True)
    generated_df = generated_log[all_columns].reset_index(drop=True)

    # Validate same columns and column types
    if (original_df.dtypes != generated_df.dtypes).any():
        raise ValueError('Mismatching dtypes')

    generated_df_of_difference_columns = generated_df[list(difference_columns)]
    dtypes = generated_df_of_difference_columns.dtypes
    invalid_dtypes = list(generated_df_of_difference_columns.columns[dtypes != 'float'])
    if len(invalid_dtypes):
        raise ValueError(f"All of identification columns must be converted to float. These identification columns have invalid dtype: {invalid_dtypes}")

    difference_weights = pd.Series(difference_weights)

    scores = []
    for generated_event_index, generated_event in generated_df.iterrows():
        score, column_scores, best_score_index = find_best_score(original_df, generated_event, identification_columns, difference_weights)

        # print()
        # print()
        # print()
        # print("event")
        # print(generated_event)

        # print()
        # print("best score index")
        # print(best_score_index)

        # print()
        # print("matched event")
        # print(original_df.iloc[best_score_index])

        scores.append(score)

    mean_score = pd.Series(scores).mean()

    return mean_score

def convert_timestamp_to_seconds_since_first(df: pd.DataFrame, columns=['time:timestamp']):
    new_df = df.copy(deep=True)
    for column in columns:
        new_df[column] = (new_df[column] - new_df[column].min()).dt.total_seconds()
    return new_df

processed_original_df = convert_timestamp_to_seconds_since_first(log.df)
processed_generated_df = convert_timestamp_to_seconds_since_first(generated_log.traces[0].df)

count_best_match_numerical(
    original_log=processed_original_df,
    generated_log=processed_generated_df,
    identification_columns=['concept:name', 'lifecycle:transition'],
    difference_weights={'time:timestamp': 1.0}
)


# print(processed_generated_df.iloc[0])

# score, normalized_column_diffs, best_score_index = find_best_score(
#     processed_original_df,
#     processed_generated_df.iloc[0],
#     identification_columns=['concept:name', 'lifecycle:transition'],
#     difference_weights={'time:timestamp': 1.0}
# )

# print(score)
# print()
# print(normalized_column_diffs.iloc[best_score_index])
# print()
# print(processed_original_df.iloc[best_score_index])




event
concept:name            incoming claim
lifecycle:transition          complete
time:timestamp                0.048564
Name: 0, dtype: object

best score index
0

matched event
concept:name            incoming claim
lifecycle:transition          complete
time:timestamp                     0.0
Name: 0, dtype: object



event
concept:name            S check if sufficient information is available
lifecycle:transition                                             start
time:timestamp                                                     0.0
Name: 1, dtype: object

best score index
10

matched event
concept:name            S check if sufficient information is available
lifecycle:transition                                             start
time:timestamp                                                     0.0
Name: 10, dtype: object



event
concept:name            S check if sufficient information is available
lifecycle:transition                                          complete
time:ti

np.float64(0.0002794820067868165)

In [249]:
generated_log.traces[0].df.dtypes['time:timestamp'].__class__

pandas.core.dtypes.dtypes.DatetimeTZDtype

In [233]:
dtypes = generated_log.df.dtypes
list(generated_log.df.columns[dtypes != 'object'])

['case:concept:name', 'time:timestamp']

In [209]:
set([1, 2]) | set([1])

{1, 2}

In [203]:
df = pd.DataFrame([[1, 1], [2, 2.1], [3, 3.1]], columns=['a', 'b'])
# s = pd.Series({ 'a': 1, 'b': 1.1 })
s = pd.Series({ 'b': 1, 'a': 1.1 })
# df - s

mul = pd.Series({ 'b': 1000, 'a': 10000 })
((df - s) * mul).abs()

,a,b
0,1000.0,0.0
1,9000.0,1100.0
2,19000.0,2100.0


In [191]:
original_df['concept:name'] == 'incoming claim'
# original_df[original_df['concept:name'] == 'incoming claim']

0         True
1        False
2        False
3        False
4        False
         ...  
46133    False
46134    False
46135    False
46136    False
46137    False
Name: concept:name, Length: 46138, dtype: bool